In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from tqdm import tqdm
from sklearn.metrics import f1_score ,roc_auc_score
import time
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

In [18]:
path = '/Data2/folder_ti/data/'
train_csv = '1st_1011_train.csv'

train = pd.read_csv(path+train_csv,index_col=0)
train = train.astype({'gender':'object'})


In [3]:
path = '/Data2/folder_ti/data/'
second_csv = 'tr_2nd_classi_after_preprocessing.csv'

second = pd.read_csv(path+second_csv,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
second.reset_index(drop=False,inplace=True)

In [21]:
second.purpose.value_counts()

��Ȱ��           2795124
��ȯ����         1362361
����ڱ�           187750
��Ÿ              138788
������������      97304
����              89915
���ñ���           72088
�ڵ�������         19750
Name: purpose, dtype: int64

In [24]:
second.drop(['purpose','income_type','rehabilitation','houseown_type','employment_type'],axis=1,inplace=True)
second = pd.merge(second, train.loc[:,['application_id','purpose','income_type','rehabilitation','houseown_type','employment_type']], on='application_id', how='left')
second.reset_index(drop=True, inplace=True)

In [25]:
second.purpose.value_counts()

생활비       2795124
대환대출      1362361
사업자금       187750
기타         138788
전월세보증금      97304
투자          89915
주택구입        72088
자동차구입       19750
Name: purpose, dtype: int64

In [7]:
second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763080 entries, 0 to 4763079
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   application_id              int64  
 1   loanapply_insert_time       object 
 2   bank_id                     int64  
 3   product_id                  int64  
 4   loan_limit                  float64
 5   loan_rate                   float64
 6   is_applied                  int64  
 7   user_id                     int64  
 8   age                         int64  
 9   gender                      int64  
 10  insert_time                 object 
 11  credit_score                int64  
 12  yearly_income               float64
 13  income_type                 object 
 14  company_year                int64  
 15  employment_type             object 
 16  houseown_type               object 
 17  desired_amount              float64
 18  purpose                     object 
 19  rehabilitation       

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728862 entries, 83603 to 518912
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   application_id             728862 non-null  int64  
 1   user_id                    728862 non-null  int64  
 2   purpose                    728862 non-null  object 
 3   is_applied                 728862 non-null  object 
 4   past.is_applied            728862 non-null  int64  
 5   m_past.is_applied          728862 non-null  float64
 6   seq                        728862 non-null  float64
 7   min.loan_rate              728862 non-null  float64
 8   product_n                  728862 non-null  int64  
 9   n.limit_over_desire        728862 non-null  int64  
 10  loan_limit_min.rate        728862 non-null  float64
 11  age                        728862 non-null  int64  
 12  gender                     728862 non-null  object 
 13  credit_score             

In [10]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  user_id / 0
col name & nan :  purpose / 0
col name & nan :  is_applied / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  min.loan_rate / 0
col name & nan :  product_n / 0
col name & nan :  n.limit_over_desire / 0
col name & nan :  loan_limit_min.rate / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  diff / 0
col name & nan :  diff2 / 0
col name & nan :  limit_over_desire_product / 0
col name & nan :  yd / 0


In [27]:
train = second

In [28]:
train.is_applied.value_counts()

0    4208369
1     554711
Name: is_applied, dtype: int64

In [29]:
data_0 = train[train.is_applied == 0]
data_1 = train[train.is_applied == 1]
target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))

/tmp/ipykernel_5563/2107596761.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
/tmp/ipykernel_5563/2107596761.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))


In [30]:
train = pd.concat([data_0,data_1], ignore_index=True)
train = train.sample(frac=1).reset_index(drop=True)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763080 entries, 0 to 4763079
Data columns (total 30 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   application_id              int64  
 1   loanapply_insert_time       object 
 2   bank_id                     int64  
 3   product_id                  int64  
 4   loan_limit                  float64
 5   loan_rate                   float64
 6   is_applied                  int64  
 7   user_id                     int64  
 8   age                         int64  
 9   gender                      int64  
 10  insert_time                 object 
 11  credit_score                int64  
 12  yearly_income               float64
 13  income_type                 object 
 14  company_year                int64  
 15  employment_type             object 
 16  houseown_type               object 
 17  desired_amount              float64
 18  purpose                     object 
 19  rehabilitation       

### c('application_id','loanapply_insert_time','user_id','insert_time'
        ,'company_enter_month','n_applied','company_year','employment_type','houseown_type'
        ,'gender','rehabilitation','yearly_income','age','existing_loan_cnt','product_id'
        ,'personal_rehabilitation_yn','income_type','personal_rehabilitation_complete_yn')

In [31]:
cat_col = ['purpose','gender','income_type','employment_type','houseown_type','rehabilitation']

In [32]:
cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(train[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i

In [33]:
cat_trans

{'purpose': {'자동차구입': 0,
  '주택구입': 1,
  '전월세보증금': 2,
  '생활비': 3,
  '사업자금': 4,
  '대환대출': 5,
  '투자': 6,
  '기타': 7},
 'gender': {0: 0, 1: 1, -1: 2},
 'income_type': {'EARNEDINCOME2': 0,
  'PRIVATEBUSINESS': 1,
  'FREELANCER': 2,
  'PRACTITIONER': 3,
  'OTHERINCOME': 4,
  'EARNEDINCOME': 5},
 'employment_type': {'기타': 0, '계약직': 1, '일용직': 2, '정규직': 3},
 'houseown_type': {'기타가족소유': 0, '전월세': 1, '배우자': 2, '자가': 3},
 'rehabilitation': {0: 0, 1: 1, 2: 2, -1: 3}}

In [34]:
from tqdm import tqdm
for cat in cat_col :
    if cat == 'gender' :
        continue
    for k in tqdm(cat_trans[cat].keys()) :
        train[cat][train[cat]==k] = cat_trans[cat][k]   

  0%|                                                                                                                       | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_5563/4081748353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
  0%|                                                                                                                       | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_5563/4081748353.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_5563/4081748353.py:6: SettingWithCopyWarning: 
A value is try

In [35]:
train_indices = train[train.set=="train"].index
valid_indices = train[train.set=="valid"].index

In [37]:
train.drop(['index'],axis=1,inplace=True)

In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763080 entries, 0 to 4763079
Data columns (total 30 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   application_id              int64  
 1   loanapply_insert_time       object 
 2   bank_id                     int64  
 3   product_id                  int64  
 4   loan_limit                  float64
 5   loan_rate                   float64
 6   is_applied                  int64  
 7   user_id                     int64  
 8   age                         int64  
 9   gender                      int64  
 10  insert_time                 object 
 11  credit_score                int64  
 12  yearly_income               float64
 13  company_year                int64  
 14  desired_amount              float64
 15  existing_loan_cnt           float64
 16  n_product                   int64  
 17  rate_rank                   float64
 18  diff                        float64
 19  rate_diff_with.min   

### c('application_id','loanapply_insert_time','user_id','insert_time'
        ,'company_enter_month','n_applied','company_year','employment_type','houseown_type'
        ,'gender','rehabilitation','yearly_income','age','existing_loan_cnt','product_id'
        ,'personal_rehabilitation_yn','income_type','personal_rehabilitation_complete_yn')

In [36]:
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id','loanapply_insert_time','insert_time','gender','yearly_income',
             'company_year','existing_loan_cnt','product_id','income_type']
cat_col = ['purpose','bank_id','rehabilitation','employment_type','houseown_type']

features = [ col for col in train.columns if col not in unused_col+[target]] 

In [ ]:
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id','loanapply_insert_time','insert_time','gender','yearly_income',
             'company_year','existing_loan_cnt','product_id','income_type']
cat_col = ['purpose','bank_id','rehabilitation','employment_type','houseown_type']
categorical_dims =  {}
for col in train.columns:
    if col in cat_col :
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train.fillna(train.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

In [38]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,set
0,1345430,409465,7,1,1,0.666667,3.0,9.2,4,3,...,0,8000000.0,3,1,0,13000000.0,1.625000,0.750000,0.266667,train
1,1340543,789104,7,1,1,1.000000,4.0,11.1,2,2,...,0,15000000.0,3,1,17,2000000.0,0.133333,1.000000,0.600000,train
2,1519681,576002,2,1,0,0.000000,1.0,15.4,5,0,...,2,64500000.0,3,8,8,-49500000.0,-0.767442,0.000000,1.131579,train
3,1488854,30828,7,0,1,0.750000,4.0,6.7,3,0,...,0,30000000.0,3,6,8,-22000000.0,-0.733333,0.000000,0.545455,valid
4,415064,398837,7,0,-1,0.465641,0.0,6.7,47,15,...,2,10000000.0,3,2,11,-1000000.0,-0.100000,0.716667,0.303030,valid


In [39]:
features

['purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'min.loan_rate',
 'product_n',
 'n.limit_over_desire',
 'loan_limit_min.rate',
 'age',
 'gender',
 'credit_score',
 'yearly_income',
 'income_type',
 'company_year',
 'employment_type',
 'houseown_type',
 'desired_amount',
 'rehabilitation',
 'existing_loan_cnt',
 'logdata',
 'diff',
 'diff2',
 'limit_over_desire_product',
 'yd']

In [40]:

X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

In [41]:
import torch
DEVICE = 'cuda:4' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda:4


In [42]:

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=20 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:4
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.44928 | val_0_auc: 0.89095 |  0:01:40s
epoch 1  | loss: 0.4008  | val_0_auc: 0.90379 |  0:03:25s
epoch 2  | loss: 0.38749 | val_0_auc: 0.906   |  0:05:09s
epoch 3  | loss: 0.38325 | val_0_auc: 0.90848 |  0:06:51s
epoch 4  | loss: 0.38541 | val_0_auc: 0.90635 |  0:08:35s
epoch 5  | loss: 0.37388 | val_0_auc: 0.90965 |  0:10:14s
epoch 6  | loss: 0.36791 | val_0_auc: 0.91615 |  0:11:57s
epoch 7  | loss: 0.3638  | val_0_auc: 0.91634 |  0:13:44s
epoch 8  | loss: 0.36119 | val_0_auc: 0.91898 |  0:15:29s
epoch 9  | loss: 0.3595  | val_0_auc: 0.9191  |  0:17:10s
epoch 10 | loss: 0.37052 | val_0_auc: 0.9169  |  0:18:50s
epoch 11 | loss: 0.36082 | val_0_auc: 0.91804 |  0:20:31s
epoch 12 | loss: 0.35816 | val_0_auc: 0.91791 |  0:22:11s
epoch 13 | loss: 0.35734 | val_0_auc: 0.91769 |  0:23:51s
epoch 14 | loss: 0.35625 | val_0_auc: 0.91967 |  0:25:31s
epoch 15 | loss: 0.35639 | val_0_auc: 0.91978 |  0:27:10s
epoch 16 | loss: 0.35613 | val_0_auc: 0.91904 |  0:28:51s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  2288.295449256897
Valid F1 score: 0.5172003351997054
Valid ROC AUC Score: 0.8387879715122495


In [43]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.6100742367210372
0.8


# 2nd Classification

In [44]:
pred_valid = thres_optimization(0.34,pred_prob_valid)

In [50]:
valid_set = train.loc[valid_indices,features+['application_id']]
valid_set['pred'] = pred_valid

In [51]:
valid_set.head()

,purpose,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,loan_limit_min.rate,age,gender,...,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,application_id,pred
3,7,1,0.750000,4.0,6.7,3,0,8000000.0,43,2,...,30000000.0,3,6,8,-22000000.0,-0.733333,0.000000,0.545455,1488854,1.0
4,7,-1,0.465641,0.0,6.7,47,15,9000000.0,35,2,...,10000000.0,3,2,11,-1000000.0,-0.100000,0.716667,0.303030,415064,1.0
5,7,1,0.250000,4.0,16.8,2,0,30000000.0,50,2,...,64500000.0,3,5,4,-34500000.0,-0.534884,0.000000,0.750000,240033,-0.0
10,2,-1,0.465641,0.0,6.5,17,0,12000000.0,44,1,...,30000000.0,3,2,17,-18000000.0,-0.600000,0.000000,1.428571,1090213,1.0
11,2,1,0.500000,2.0,12.5,6,6,19000000.0,27,1,...,10000000.0,3,2,0,9000000.0,0.900000,1.000000,0.263158,666684,1.0


In [52]:
second.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above
1,1,62,200,8000000.0,14.8,0,1.0,0.0,0.00
2,1,38,223,19000000.0,17.1,0,2.5,2.3,1.35
3,1,38,16,19000000.0,17.1,0,2.5,2.3,0.00
4,1,59,150,19000000.0,17.7,0,4.0,2.9,0.60
5,1,37,206,32000000.0,18.9,0,5.0,4.1,1.20


In [53]:
train_2nd = pd.merge(second, valid_set, on='application_id', how='left')
train_2nd.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,pred
0,1,62,200,8000000.0,14.8,0,1.0,0.0,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,38,223,19000000.0,17.1,0,2.5,2.3,1.35,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,38,16,19000000.0,17.1,0,2.5,2.3,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,59,150,19000000.0,17.7,0,4.0,2.9,0.60,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,37,206,32000000.0,18.9,0,5.0,4.1,1.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for col in train_2nd.columns :
    print('col name & nan : ', col,'/',train_2nd[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  rate_rank / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  purpose / 5123908
col name & nan :  past.is_applied / 5123908
col name & nan :  m_past.is_applied / 5123908
col name & nan :  seq / 5123908
col name & nan :  min.loan_rate / 5123908
col name & nan :  product_n / 5123908
col name & nan :  n.limit_over_desire / 5123908
col name & nan :  loan_limit_min.rate / 5123908
col name & nan :  age / 5123908
col name & nan :  gender / 5123908
col name & nan :  credit_score / 5123908
col name & nan :  yearly_income / 5123908
col name & nan :  income_type / 5123908
col name & nan :  company_year / 5123908
col name & nan :  employment_type / 5123908
col name & nan :  houseown_type / 5123908
col name & nan :  desired_amount / 

In [55]:
train_2nd.dropna(axis=0, how='any',inplace=True)

In [69]:
train_2nd.nunique()

application_id               364473
bank_id                          62
product_id                      165
loan_limit                       70
loan_rate                       173
is_applied                        2
rate_rank                       155
rate_diff_with.min              194
rate_diff_with.above             41
purpose                           8
past.is_applied                   3
m_past.is_applied              1307
seq                              14
product_n                        47
n.limit_over_desire              16
age                              72
gender                            3
credit_score                     78
yearly_income                    87
income_type                       6
company_year                     11
employment_type                   4
houseown_type                     4
desired_amount                   66
rehabilitation                    4
existing_loan_cnt                12
logdata                          35
diff                        

In [62]:
train_2nd.iloc[:,:20].head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,loan_limit_min.rate,age,gender,credit_score
6,2,62,200,15000000.0,9.3,0,1.0,0.0,0.00,7.0,1.0,1.000000,6.0,9.3,1.0,1.0,15000000.0,42.0,2.0,590.0
7,6,29,265,20000000.0,9.4,0,1.0,0.0,0.00,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
8,6,21,196,23000000.0,12.5,0,2.0,3.1,1.35,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
9,6,64,254,33000000.0,13.2,0,3.0,3.8,0.70,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0
10,6,26,80,38000000.0,13.6,0,4.0,4.2,0.40,2.0,1.0,0.333333,6.0,9.4,15.0,3.0,20000000.0,32.0,2.0,630.0


In [63]:
train_2nd.drop(['loan_limit_min.rate','min.loan_rate'],axis=1,inplace=True)

In [65]:
train_2nd.to_csv('/Data2/folder_ti/data/train_2nd.csv',index=False)

In [3]:
path = '/Data2/folder_ti/data/'
train_csv = 'train_2nd.csv'

train_2nd = pd.read_csv(path+train_csv,index_col=0)



/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
train_2nd.reset_index(drop=False,inplace=True)

In [6]:
train_2nd.head()

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,rate_rank,rate_diff_with.min,rate_diff_with.above,purpose,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,pred
0,2,59,119,15000000.0,9.3,0,1.0,0.0,0.00,7,...,2,10000000.0,3,11.0,11.0,5000000.0,0.500000,1.0,0.232558,1.0
1,6,28,160,20000000.0,9.4,0,1.0,0.0,0.00,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
2,6,20,117,23000000.0,12.5,0,2.0,3.1,1.35,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
3,6,61,152,33000000.0,13.2,0,3.0,3.8,0.70,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0
4,6,25,45,38000000.0,13.6,0,4.0,4.2,0.40,2,...,0,36000000.0,2,4.0,0.0,-16000000.0,-0.444444,0.2,1.000000,1.0


In [7]:
train_2nd.columns

Index(['application_id', 'bank_id', 'product_id', 'loan_limit', 'loan_rate',
       'is_applied', 'rate_rank', 'rate_diff_with.min', 'rate_diff_with.above',
       'purpose', 'past.is_applied', 'm_past.is_applied', 'seq', 'product_n',
       'n.limit_over_desire', 'age', 'gender', 'credit_score', 'yearly_income',
       'income_type', 'company_year', 'employment_type', 'houseown_type',
       'desired_amount', 'rehabilitation', 'existing_loan_cnt', 'logdata',
       'diff', 'diff2', 'limit_over_desire_product', 'yd', 'pred'],
      dtype='object')

In [8]:
train_2nd.drop(['age','gender','income_type','employment_type','houseown_type','yearly_income','existing_loan_cnt'],axis=1,inplace=True)

In [9]:
#nunique = train.nunique()
types = train_2nd.dtypes
target = 'is_applied'
unused_col = ['set','application_id','pred']
#cat_col = ['bank_id','product_id','purpose','income_type','rehabilitation','gender','employment_type','houseown_type']
cat_col = ['bank_id','product_id','purpose','rehabilitation']
categorical_dims =  {}
for col in train_2nd.columns:
    if col in cat_col :
        print(col, train_2nd[col].nunique())
        l_enc = LabelEncoder()
        train_2nd[col] = l_enc.fit_transform(train_2nd[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train_2nd.fillna(train_2nd.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train_2nd.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

bank_id 62
product_id 165
purpose 8
rehabilitation 4


In [10]:
features

['bank_id',
 'product_id',
 'loan_limit',
 'loan_rate',
 'rate_rank',
 'rate_diff_with.min',
 'rate_diff_with.above',
 'purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'product_n',
 'n.limit_over_desire',
 'credit_score',
 'company_year',
 'desired_amount',
 'rehabilitation',
 'logdata',
 'diff',
 'diff2',
 'limit_over_desire_product',
 'yd']

In [11]:
data_0 = train_2nd[train_2nd.is_applied == 0]
data_1 = train_2nd[train_2nd.is_applied == 1]
target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))

/tmp/ipykernel_4599/2215527724.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
/tmp/ipykernel_4599/2215527724.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))


In [12]:
train_2nd = pd.concat([data_0,data_1], ignore_index=True)
train_2nd = train_2nd.sample(frac=1).reset_index(drop=True)

In [13]:
train_indices = train_2nd[train_2nd.set=="train"].index
valid_indices = train_2nd[train_2nd.set=="valid"].index
test_indices = train_2nd[train_2nd.pred==1].index
test_else_indices = train_2nd[train_2nd.pred==0].index

X_train = train_2nd[features].values[train_indices]
y_train = train_2nd[target].values[train_indices]

X_valid = train_2nd[features].values[valid_indices]
y_valid = train_2nd[target].values[valid_indices]

X_test = train_2nd[features].values[test_indices]
y_test = train_2nd[target].values[test_indices]

test_else_set = train_2nd.loc[test_else_indices,:]
test_set = train_2nd.loc[test_indices,:]

In [14]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=30 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:2


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.41493 | val_0_auc: 0.90266 |  0:01:20s
epoch 1  | loss: 0.38231 | val_0_auc: 0.90984 |  0:02:38s
epoch 2  | loss: 0.36867 | val_0_auc: 0.91337 |  0:03:56s
epoch 3  | loss: 0.35936 | val_0_auc: 0.91865 |  0:05:17s
epoch 4  | loss: 0.3519  | val_0_auc: 0.92103 |  0:06:36s
epoch 5  | loss: 0.35116 | val_0_auc: 0.922   |  0:07:53s
epoch 6  | loss: 0.34539 | val_0_auc: 0.91345 |  0:09:11s
epoch 7  | loss: 0.34731 | val_0_auc: 0.92325 |  0:10:30s
epoch 8  | loss: 0.34245 | val_0_auc: 0.92389 |  0:11:47s
epoch 9  | loss: 0.34148 | val_0_auc: 0.92463 |  0:13:04s
epoch 10 | loss: 0.3394  | val_0_auc: 0.92505 |  0:14:22s
epoch 11 | loss: 0.33914 | val_0_auc: 0.92533 |  0:15:41s
epoch 12 | loss: 0.33869 | val_0_auc: 0.92509 |  0:16:59s
epoch 13 | loss: 0.33839 | val_0_auc: 0.9259  |  0:18:16s
epoch 14 | loss: 0.33726 | val_0_auc: 0.92603 |  0:19:35s
epoch 15 | loss: 0.33677 | val_0_auc: 0.92608 |  0:20:53s
epoch 16 | loss: 0.33642 | val_0_auc: 0.92618 |  0:22:10s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  2609.6403727531433
Valid F1 score: 0.3580792404601059
Valid ROC AUC Score: 0.853156572337846


## 0.34, 0.85

In [15]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.4765324595658093
0.82


In [16]:
pred_prob_test = clf.predict_proba(X_test)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_test = [0]
best_thres_test = 0
for i in range(1,100) :
    pre_test = thres_optimization(i/100,pred_prob_test)
    
    f1_test = f1_score(y_test, pre_test)
    
    score_test.append(f1_test)
    
best_thres_test = score_test.index(max(score_test))/100
print(max(score_test))
print(best_thres_test)

0.4964737930686575
0.83


In [17]:
test_else_set['pred2'] = 0
test_set['pred2'] = thres_optimization(0.85,pred_prob_test)

In [18]:
final_test = pd.concat([test_else_set,test_set], ignore_index=True)

In [19]:
print('2nd_classification_f1_score : ',f1_score(final_test.is_applied.values, final_test.pred2.values))

2nd_classification_f1_score :  0.4703220347453553


In [20]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(final_test.is_applied.values, final_test.pred2.values))

[[4696658  165833]
 [ 141528  136459]]


## Smote 2nd_classification

In [20]:
from imblearn.over_sampling import SMOTE

smote_sample = SMOTE(random_state=0)
X_train_over,y_train_over = smote_sample.fit_resample(X_train,y_train)

In [21]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train_over, y_train=y_train_over,eval_set=[(X_valid, y_valid)],
    max_epochs=20 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:2


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.33782 | val_0_auc: 0.87443 |  0:02:32s
epoch 1  | loss: 0.20629 | val_0_auc: 0.89427 |  0:05:04s
epoch 2  | loss: 0.15115 | val_0_auc: 0.90206 |  0:07:37s
epoch 3  | loss: 0.12415 | val_0_auc: 0.89792 |  0:10:23s
epoch 4  | loss: 0.12376 | val_0_auc: 0.90591 |  0:13:06s
epoch 5  | loss: 0.11828 | val_0_auc: 0.90822 |  0:15:38s
epoch 6  | loss: 0.1126  | val_0_auc: 0.89729 |  0:18:10s
epoch 7  | loss: 0.11675 | val_0_auc: 0.89843 |  0:20:44s
epoch 8  | loss: 0.11173 | val_0_auc: 0.9025  |  0:23:38s
epoch 9  | loss: 0.11051 | val_0_auc: 0.88563 |  0:26:47s
epoch 10 | loss: 0.10906 | val_0_auc: 0.89408 |  0:29:53s
epoch 11 | loss: 0.10761 | val_0_auc: 0.82355 |  0:32:51s
epoch 12 | loss: 0.10593 | val_0_auc: 0.86287 |  0:36:31s
epoch 13 | loss: 0.10443 | val_0_auc: 0.89813 |  0:39:47s
epoch 14 | loss: 0.1047  | val_0_auc: 0.88459 |  0:43:02s
epoch 15 | loss: 0.10338 | val_0_auc: 0.86952 |  0:46:20s
epoch 16 | loss: 0.10434 | val_0_auc: 0.86698 |  0:49:38s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  4711.617521524429
Valid F1 score: 0.40708203031032003
Valid ROC AUC Score: 0.6769016812586578


In [22]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

0.4350965878317386
0.35


In [23]:
pred_prob_test = clf.predict_proba(X_test)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_test = [0]
best_thres_test = 0
for i in range(1,100) :
    pre_test = thres_optimization(i/100,pred_prob_test)
    
    f1_test = f1_score(y_test, pre_test)
    
    score_test.append(f1_test)
    
best_thres_test = score_test.index(max(score_test))/100
print(max(score_test))
print(best_thres_test)

0.4581477959864
0.34


In [28]:
test_else_set['pred2'] = 0
test_set['pred2'] = thres_optimization(0.34,pred_prob_test)

In [29]:
final_test = pd.concat([test_else_set,test_set], ignore_index=True)

In [30]:
print('2nd_classification_f1_score : ',f1_score(final_test.is_applied.values, final_test.pred2.values))

2nd_classification_f1_score :  0.4380195750480084


In [31]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(final_test.is_applied.values, final_test.pred2.values))

[[4636177  226314]
 [ 136568  141419]]
